# 0. Import Required Libraries

In [1]:
%matplotlib inline

import mlflow
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

import math
from lib.utils import *

# 1. Define Classifier Architecture

In [2]:
class Classifier(nn.Module):
    def __init__(self, channel, in_len):
        super(Classifier, self).__init__()
        
        self.fc1_size = channel * in_len ** 2
        self.fc2_size = self.fc1_size * 2
        self.fc3_size = self.fc2_size
        self.fc4_size = self.fc1_size
        
        self.fc1 = nn.Linear(self.fc1_size, self.fc2_size)
        self.fc2 = nn.Linear(self.fc2_size, self.fc3_size)
        self.fc3 = nn.Linear(self.fc3_size, self.fc4_size)
        self.fc4 = nn.Linear(self.fc4_size, 2)
        
        self.dropout = nn.Dropout(p=0.3)
    def forward(self, x):
        # Flatten input
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        
        x = self.dropout(F.relu(self.fc2(x)))
        
        x = self.dropout(F.relu(self.fc3(x)))
        
        x = F.log_softmax(self.fc4(x), dim=1)
        return x
    
    def train_network(self, trainloader, val_loader, epochs=20):
        pass
                    
    def test(self):
        pass

# 2. Start MlFlow Run

In [3]:
mlflow.set_tracking_uri("file:.\mlruns")
mlflow.start_run()

params = {}
artifacts = []
metrics = {}

# 3. Load Data

In [4]:
data, filenames = load_data(10, "./data/modis")

In [ ]:
labels = [data[i][0] for i in range(len(data))]
train_data = [data[i][1:] for i in range(len(data))]

# 4. Chunk Images
Each image is broken up into bx9x3x3 tensors, where b is the batch size. 

In [ ]:
%%time

print("{:30} shape: (batch, channel, height, width)".format("filename"))

chunk_sum = 0

#datasets = []

for index, image in enumerate(train_data):
    chunked_image = chunk_image(merge_dims(image))
    #chunked_label = chunk_image(labels[index], label=True)
    
    chunk_sum += chunked_image.shape[0]
    
    #datasets.append(torch.utils.data.TensorDataset(torch.from_numpy(chunked_image), torch.from_numpy(chunked_label)))
    
    print("{:30} shape: {}".format(filenames[index], chunked_image.shape))
    
print("\nTotal {} x {} chunks: {}".format(chunked_image.shape[-1], chunked_image.shape[-1], chunk_sum))

filename                       shape: (batch, channel, height, width)
arkansas_city.tif              shape: (556850, 9, 3, 3)
assiniboine.tif                shape: (2182830, 9, 3, 3)
bay_area.tif                   shape: (3165612, 9, 3, 3)
berkeley.tif                   shape: (431616, 9, 3, 3)
kashmore.tif                   shape: (3304125, 9, 3, 3)
kashmore_north.tif             shape: (328040, 9, 3, 3)
katrina.tif                    shape: (1263893, 9, 3, 3)
katrina_slidell.tif            shape: (249676, 9, 3, 3)
malawi.tif                     shape: (423504, 9, 3, 3)
mississippi_june.tif           shape: (824680, 9, 3, 3)
mississippi_may.tif            shape: (824680, 9, 3, 3)
parana.tif                     shape: (686907, 9, 3, 3)
sava.tif                       shape: (753087, 9, 3, 3)
sava_west.tif                  shape: (410116, 9, 3, 3)
unflooded_mississippi.tif      shape: (1662630, 9, 3, 3)
unflooded_new_orleans.tif      shape: (1403010, 9, 3, 3)

Total 3 x 3 chunks: 1847125

In [ ]:
i = 0
params["train_image"] = filenames[i]

chunked_data = chunk_image(merge_dims(train_data[i]))
chunked_labels = chunk_image(labels[i], label=True)
chunked_labels.shape

(556850,)

In [ ]:
batch_size = 2048
params["batch_size"] = batch_size

trainloader, val_loader = split_trainset(chunked_data, chunked_labels, ratio=0.7, batch_size=batch_size)

# 5. Instantiate Model and Optimizer

In [ ]:
model = Classifier(9, 3)

In [ ]:
#model = parallelize(model)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# 6. Use GPU, if available

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
model.to(device)

Classifier(
  (fc1): Linear(in_features=81, out_features=162, bias=True)
  (fc2): Linear(in_features=162, out_features=162, bias=True)
  (fc3): Linear(in_features=162, out_features=81, bias=True)
  (fc4): Linear(in_features=81, out_features=2, bias=True)
  (dropout): Dropout(p=0.3)
)

# 7. Train and Validate Model

In [ ]:
epochs = 70
params["epochs"] = epochs

train_losses = []
val_losses = []
val_accuracies = []

min_val_loss = float("inf")

for epoch in range(epochs):
    model.train()
    train_loss = 0
    
    for batch, ground_truth in trainloader:
        # ============================================
        #            TRAINING
        # ============================================
        batch, ground_truth = batch.to(device), ground_truth.to(device)
        output = model.forward(batch.float())
        # Clear gradients in optimizer
        optimizer.zero_grad()
        # Calculate loss
        loss = criterion(output.squeeze(), ground_truth.long())
        train_loss += loss.item()
        # Backpropagation
        loss.backward()
        # Update weights
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            val_loss = 0
            
            y_pred = np.array([])
            y_true = np.array([])
            
            for batch, ground_truth in val_loader:
                # ============================================
                #            VALIDATION
                # ============================================
                batch, ground_truth = batch.to(device), ground_truth.to(device)
                # forward pass
                log_probs = model.forward(batch.float())
                probs = torch.exp(log_probs)
                
                top_p, top_class = probs.topk(1, dim=1)
                y_pred = np.append(y_pred, cuda_to_numpy(top_class))
                y_true = np.append(y_true, cuda_to_numpy(ground_truth))
                
                # calculate loss
                loss = criterion(log_probs.squeeze(), ground_truth.long())
                val_loss += loss.item()

    # Print epoch summary
    t_loss_avg = train_loss / len(trainloader)
    v_loss_avg = val_loss / len(val_loader)
    accuracy = accuracy_score(y_true, y_pred)
    
    if v_loss_avg < min_val_loss:
        torch.save(model.state_dict(), "./artifacts/model.pth")
        artifacts.append("model.pth")
        
    mlflow.log_metric("train_loss", t_loss_avg)
    mlflow.log_metric("val_loss", v_loss_avg)
    mlflow.log_metric("validation_accuracy", accuracy)
    
    train_losses.append(t_loss_avg)
    val_losses.append(v_loss_avg)
    val_accuracies.append(accuracy)
       
    print('Epoch [{:5d}/{:5d}] | train loss: {:6.4f} | validation loss: {:6.4f} | validation accuracy: {:6.4f}%'.format(
                epoch+1, epochs, t_loss_avg, v_loss_avg, accuracy * 100))

Epoch [    1/   70] | train loss: 0.2563 | validation loss: 0.1772 | validation accuracy: 92.6946%
Epoch [    2/   70] | train loss: 0.1824 | validation loss: 0.1730 | validation accuracy: 92.7641%
Epoch [    3/   70] | train loss: 0.1782 | validation loss: 0.1698 | validation accuracy: 92.7778%
Epoch [    4/   70] | train loss: 0.1752 | validation loss: 0.1679 | validation accuracy: 92.8503%
Epoch [    5/   70] | train loss: 0.1719 | validation loss: 0.1657 | validation accuracy: 92.9311%
Epoch [    6/   70] | train loss: 0.1699 | validation loss: 0.1641 | validation accuracy: 92.8880%
Epoch [    7/   70] | train loss: 0.1687 | validation loss: 0.1647 | validation accuracy: 92.8640%
Epoch [    8/   70] | train loss: 0.1680 | validation loss: 0.1607 | validation accuracy: 93.0257%
Epoch [    9/   70] | train loss: 0.1663 | validation loss: 0.1621 | validation accuracy: 93.0484%
Epoch [   10/   70] | train loss: 0.1649 | validation loss: 0.1585 | validation accuracy: 93.2920%
Epoch [   

# 8. Plot Learning Curve

In [ ]:
plt.plot(train_losses, label="Training")
plt.plot(val_losses, label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve for MODIS Image Classifier")
plt.legend()

figure_name = "train_loss.png"
plt.savefig("./artifacts/" + figure_name)
artifacts.append(figure_name)

In [ ]:
plt.plot(val_accuracies)
plt.xlabel("Epochs")
plt.ylabel("Accuracy(%)")
plt.title("Validation Accuracy for MODIS Image Classifier")

figure_name = "val_accuracy.png"
plt.savefig("./artifacts/" + figure_name)
artifacts.append(figure_name)

# 8. Wrap up MlFlow Run

In [ ]:
for name, val in params.items():
    mlflow.log_param(name, val)

for name, val in metrics.items():
    mlflow.log_metric(name, val)
    
artifact_path = "./artifacts/"
for name in artifacts:
    mlflow.log_artifact(artifact_path + name)

mlflow.end_run()